# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [105]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
warnings.filterwarnings('ignore')

<b> Изучение данных </b>

In [106]:
data = pd.read_csv("/datasets/autos.csv")
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [107]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

<b> Признаки </b>
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

<b> Целевой признак </b>
- Price — цена (евро)

In [108]:
data[data.duplicated(keep=False)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
41529,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15
88087,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
90964,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
187735,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


In [109]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


<b> Важные признаки: </b> VehicleType, Gearbox, Power, Kilometer, FuelType, Brand, NotRepaired, RegistrationYear, Model.

Признаки NumberOfPictures, PostalCode не несут полезной информации, их можно удалить.

DateCrawled, LastSeen, NumberOfPictures могли быть полезными для прогнозирования скорости продажи, но для прогнозирования цены авто они не нужны.

DateCreated может быть полезна для анализа с учетом инфляции.

По многим важным параметрам есть пропуски и нулевые значеня необходимо будет восстановить их или удалить.
Есть неадекватные значения параметров необходимо будет восстановить их или удалить
Найдено небольшое количество дубликатов необходимо их удалить

<b> Предобработка данных </b>

Обраотаем целевой признак price, пропусков данных нет, но есть цена равная нулю.  Восстановить по среднему было бы некоректно, так как признак целевой и это напрямую повлияет на прогноз.

In [110]:
print("Колличество объявлений с нулевой ценой:",len(data.loc[data['Price'] == 0]))

Колличество объявлений с нулевой ценой: 10772


In [111]:
data = data.loc[data['Price'] != 0]

Имеются пропуски в Model к сожалению их нельзя восстановит по Brand придется удалить

In [112]:
print("Колличество объявлений с пропущенной моделью:", len(data.loc[data['Model'].isna()]))

Колличество объявлений с пропущенной моделью: 17521


In [113]:
data = data.loc[~data['Model'].isna()]

Колличество пропусков в VehicleType около 10 процентов, данных слишком много, чтобы просто от них избавится, если заменить на среднюю это тоже может сказаться на точности предсказания. Если брать в расчет, что в дальнейшем пользователи при оценке автомобиля могут так же не вводить тип кузова, то стоит заменить пропущенные значения на unknown

In [114]:
print("Колличество объявлений с незаполненным типом кузова:", len(data.loc[data['VehicleType'].isna()]))

Колличество объявлений с незаполненным типом кузова: 28166


In [115]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')

Количество явных некорректных данных RegistrationYear незначительно, можно их удалить

In [116]:
print("Колличество объявлений с некорректной годом:",len(data.loc[(data['RegistrationYear'] > 2022) | (data['RegistrationYear'] < 1769)]))

Колличество объявлений с некорректной годом: 83


In [164]:
data = data.loc[(data['RegistrationYear'] <= 2016) & (data['RegistrationYear'] >= 1900)]

In [118]:
print("Максимальная дата создания объявления:", max(data['DateCreated']))

Максимальная дата создания объявления: 2016-04-07 00:00:00


Колличество пропусков в Gearbox тоже велико. Заменю на наиболее встречающийся тип коробки в модели.

In [119]:
print("Колличество объявлений с незаполненным типом коробки:", len(data.loc[data['Gearbox'].isna()]))

Колличество объявлений с незаполненным типом коробки: 14205


In [120]:
data['Gearbox'] = data['Gearbox'].fillna(data
                                         .groupby('Model')['Gearbox']
                                         .transform(lambda x: x.value_counts().idxmax())
                                        )

Имеются значения Power равные 0 и больше 1000 что являеться некорректным, можно заменить их на медиану по модели

In [121]:
print("Колличество объявлений с некорректной мощностью:",len(data.loc[(data['Power'] > 1000) | (data['Power'] <= 0)]))

Колличество объявлений с некорректной мощностью: 31519


In [19]:
data.loc[(data['Power'] > 1000) | (data['Power'] <= 0), 'Power'] = None
data['Power'] = data['Power'].fillna(data.groupby('Model')['Power'].transform('median'))
data = data.loc[~data['Power'].isna()]
data['Power'] = data['Power'].astype('int64')

Пропуски в FuelType заменю на среднее по моделям

In [122]:
data['FuelType'] = data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform(lambda x: x.value_counts().idxmax()))

Пропуски в NotRepaired состоявляют треть от данных. Скорее всего если автомобиль не был поврежден то этот параметр просто не заполнялся. Заменю на "yes"

In [123]:
data["NotRepaired"].fillna("yes", inplace=True)

Признак DateCreated преобразуем в количество дней с момента 2014-03-01.

In [124]:
print("Минимальная дата создания объявления:", min(data['DateCreated']))

Минимальная дата создания объявления: 2014-03-10 00:00:00


In [125]:
base_date = pd.Timestamp(min(data['DateCreated']))
data['DateCreated'] = data['DateCreated'].map(lambda date : (pd.Timestamp(date) - base_date).days)

Удалим дубликаты

In [126]:
data = data.drop_duplicates()

Удалю ненужные признаки NumberOfPictures, PostalCode, DateCrawled, LastSeen

In [127]:
data = data.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'LastSeen'], axis=1)

In [128]:
data = data.reset_index(drop=True)

In [129]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325986 entries, 0 to 325985
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              325986 non-null  int64 
 1   VehicleType        325986 non-null  object
 2   RegistrationYear   325986 non-null  int64 
 3   Gearbox            325986 non-null  object
 4   Power              325986 non-null  int64 
 5   Model              325986 non-null  object
 6   Kilometer          325986 non-null  int64 
 7   RegistrationMonth  325986 non-null  int64 
 8   FuelType           325986 non-null  object
 9   Brand              325986 non-null  object
 10  NotRepaired        325986 non-null  object
 11  DateCreated        325986 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 29.8+ MB


In [130]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,DateCreated
count,325986.000000,325986.000000,325986.000000,325986.000000,325986.000000,325986.000000
mean,4623.371991,2003.268070,111.963824,128596.764892,5.853948,741.798126
std,4530.481176,7.061917,182.665855,36964.094306,3.665998,9.399254
min,1.000000,1910.000000,0.000000,5000.000000,0.000000,0.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000,734.000000
50%,2950.000000,2003.000000,105.000000,150000.000000,6.000000,742.000000
75%,6649.750000,2008.000000,143.000000,150000.000000,9.000000,750.000000
max,20000.000000,2019.000000,20000.000000,150000.000000,12.000000,759.000000


<b> Кодирование категориальных признаков </b>

Преобразую категориальные признаки с большим колличеством категорий техникой Ordinal Encoding.

In [131]:
data_reg = data

In [132]:
encoder = OrdinalEncoder()
encoder.fit(data) 
data_ordinal = pd.DataFrame(encoder.transform(data), columns=data.columns)

data_ordinal.reset_index()
data.loc[:,['Brand', 'Model']] = data_ordinal.loc[:,['Brand', 'Model']]
data['Brand'] = data['Brand'].astype(int)
data['Model'] = data['Model'].astype(int)

Признаки VehicleType, Gearbox, FuelType, NotRepaired закодируем one-hot encoding.

In [133]:
def dum(data, column):
    df = pd.get_dummies(data[column], prefix=column, drop_first=True)
    data = data.drop(column, axis=1)
    return data.join(df)

In [134]:
data = dum(data, 'VehicleType')
data = dum(data, 'Gearbox')
data = dum(data, 'FuelType')
data = dum(data, 'NotRepaired')

<b> Разобьем данные на тестовую и валидационную выборки, и на признаки и целевой признак </b>

In [135]:
target = data['Price']
features = data.drop('Price', axis=1)
features_train, features_part, target_train, target_part = train_test_split(features, target, test_size=0.30, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_part, target_part, test_size=0.5, random_state=12345)

In [136]:
print(features_train.shape, features_valid.shape, features_test.shape, target_train.shape,  target_valid.shape, target_test.shape)

(228190, 23) (48898, 23) (48898, 23) (228190,) (48898,) (48898,)


In [137]:
features_train

,RegistrationYear,Power,Model,Kilometer,RegistrationMonth,Brand,DateCreated,VehicleType_convertible,VehicleType_coupe,VehicleType_other,...,VehicleType_unknown,VehicleType_wagon,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol,NotRepaired_yes
220395,2004,88,151,125000,6,23,747,0,0,0,...,0,0,1,0,0,0,0,0,1,0
98218,2004,90,163,150000,6,31,740,0,0,0,...,0,1,1,0,1,0,0,0,0,0
222077,2010,105,62,90000,6,37,751,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4727,2003,0,83,150000,0,24,753,0,0,0,...,0,0,1,0,0,0,0,0,1,1
252181,2005,88,57,150000,6,5,751,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158838,2005,224,31,150000,4,1,742,0,0,0,...,0,1,0,0,1,0,0,0,0,0
47873,1997,320,207,150000,8,20,730,1,0,0,...,0,0,0,0,0,0,0,0,1,0
86398,2003,142,11,150000,6,2,749,0,0,0,...,0,0,1,0,0,0,0,0,1,1
77285,1994,0,59,5000,10,20,737,0,0,0,...,0,0,1,0,0,0,0,0,1,0


Стандартизирую численные признаки и преобразованные техникой Ordinal Encoding.

In [138]:
numeric = ['RegistrationYear', 'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'Brand',  'DateCreated']

In [139]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric]) 
print(features_train.head())

        RegistrationYear     Power     Model  Kilometer  RegistrationMonth  \
220395          0.101876 -0.130016  0.592702  -0.096124           0.039899   
98218           0.101876 -0.119169  0.762448   0.580666           0.039899   
222077          0.950364 -0.037818 -0.666246  -1.043631           0.039899   
4727           -0.039538 -0.607276 -0.369191   0.580666          -1.596818   
252181          0.243291 -0.130016 -0.736974   0.580666           0.039899   

           Brand  DateCreated  VehicleType_convertible  VehicleType_coupe  \
220395  0.189371     0.550637                        0                  0   
98218   0.805469    -0.190790                        0                  0   
222077  1.267543     0.974310                        0                  0   
4727    0.266383     1.186146                        0                  0   
252181 -1.196850     0.974310                        0                  1   

        VehicleType_other  ...  VehicleType_unknown  VehicleType_wag

<b> Вывод </b>

Дубликаты удалены. Отобраны признаки, необходимые для построения моделей.

Пропущенные значения, выбросы, неправдоподобные значения в признаках заполнены на основе имеющейся инфрмации или удалены.

Категориальные признаки преобразованы с помощью one-hot encoding.

Признак с большим колличеством категорий закодирован техникой Ordinal Encoding.

Данные разделены на тестовую и обучающую выборки и стандартизированны.

## Обучение моделей

<b> Linear Regression </b>

In [140]:
%%time
lmodel = LinearRegression()
lmodel.fit(features_train, target_train)

CPU times: user 289 ms, sys: 124 ms, total: 413 ms
Wall time: 398 ms


LinearRegression()

In [141]:
%%time
preds_lmodel = lmodel.predict(features_test)

CPU times: user 8.96 ms, sys: 101 µs, total: 9.07 ms
Wall time: 5.83 ms


In [142]:
mse_lmodel = mean_squared_error(target_test, preds_lmodel)
print("RMSE для линейной модели на тестовой выборке:", round((mse_lmodel) ** 0.5, 2))

RMSE для линейной модели на тестовой выборке: 3189.64


<b> Регрессия Ridge </b>

In [143]:
%%time
rmodel = Ridge()
hyperparams = [{'solver':['auto', 'svd', 'cholesky', 'lsqr','sparse_cg']}]
rmodel = GridSearchCV(rmodel, hyperparams, scoring='neg_mean_squared_error')
rmodel.fit(features_train, target_train)
print(rmodel.best_params_)

{'solver': 'svd'}
CPU times: user 7.29 s, sys: 6.39 s, total: 13.7 s
Wall time: 13.7 s


In [144]:
%%time
preds_rmodel = rmodel.predict(features_test)

CPU times: user 24.2 ms, sys: 24.3 ms, total: 48.5 ms
Wall time: 10 ms


In [43]:
mse_rmodel = mean_squared_error(target_test, preds_rmodel)
print("RMSE для Ridge модели на тестовой выборке:", round((mse_rmodel) ** 0.5, 2))

RMSE для Ridge модели на тестовой выборке: 2888.98


<b> Регрессия DecisionTreeRegresso </b>

In [145]:
%%time
trmodel = DecisionTreeRegressor(criterion='mse', 
                              max_depth=8, 
                              random_state=12345) 
trmodel.fit(features_train, target_train)

CPU times: user 621 ms, sys: 24.1 ms, total: 645 ms
Wall time: 643 ms


DecisionTreeRegressor(max_depth=8, random_state=12345)

In [146]:
%%time
preds_trmodel = trmodel.predict(features_test)

CPU times: user 9.28 ms, sys: 38 µs, total: 9.32 ms
Wall time: 7.74 ms


In [147]:
mse_trmodel = mean_squared_error(target_test, preds_trmodel)
print("RMSE для DecisionTreeRegresso модели на тестовой выборке:", round((mse_trmodel) ** 0.5, 2))

RMSE для DecisionTreeRegresso модели на тестовой выборке: 2142.78


<b> Регрессия CatBoostRegressor </b>

In [148]:
numerical_features = ['DateCreated', 'Price', 'RegistrationYear', 'Power', 'Kilometer']

In [149]:
categorical_features = [col for col in list(features_train.columns) if col not in numerical_features]

В связи с тем, что CatBoostRegressor принимает только целые числа или строки в категориях, пришлость пересобрать признаки.

Так же попробывал оставить категориальные признаки без one-hot encoding, качество немного выросло на CatBoost и LGBM, но время обработки многократно увеличилось!

In [150]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)
data_reg.loc[:,['VehicleType','Gearbox', 'Brand', 'FuelType', 'NotRepaired']] = data_ordinal.loc[:,['VehicleType','Gearbox', 'Brand', 'FuelType', 'NotRepaired']]
catfeatures = data_reg.drop('Price', axis=1)
cattarget = data_reg['Price']
catfeatures_train, catfeatures_test, cattarget_train, cattarget_test = train_test_split(features, target, test_size=0.25, random_state=12345)

In [151]:
numericcat = ['RegistrationYear', 'Power', 'Kilometer', 'DateCreated']
scaler = StandardScaler()
scaler.fit(catfeatures_train[numericcat])
catfeatures_train[numericcat] = scaler.transform(catfeatures_train[numericcat])
catfeatures_test[numericcat] = scaler.transform(catfeatures_test[numericcat]) 

In [152]:
%%time
catmodel = CatBoostRegressor(learning_rate=0.5, random_state=12345, verbose=False) 
catmodel.fit(catfeatures_train, target_train, cat_features=categorical_features)

CPU times: user 2min 18s, sys: 707 ms, total: 2min 19s
Wall time: 2min 20s


In [153]:
%%time
cat_predict = catmodel.predict(catfeatures_test)

CPU times: user 951 ms, sys: 0 ns, total: 951 ms
Wall time: 959 ms


In [154]:
mse_catmodel = mean_squared_error(target_test, cat_predict)
print("RMSE для CatBoostRegressor модели на тестовой выборке:", round((mse_catmodel) ** 0.5, 2))

RMSE для CatBoostRegressor модели на тестовой выборке: 1609.26


<b>  Регрессия LGBMRegressor </b>

In [156]:
%%time
lgbmmodel = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=100, 
                      random_state=12345,
                      cat_features=categorical_features)
lgbmmodel.fit(features_train, target_train)

CPU times: user 12.6 s, sys: 66.5 ms, total: 12.7 s
Wall time: 12.7 s


LGBMRegressor(cat_features=['Model', 'RegistrationMonth', 'Brand',
                            'VehicleType_convertible', 'VehicleType_coupe',
                            'VehicleType_other', 'VehicleType_sedan',
                            'VehicleType_small', 'VehicleType_suv',
                            'VehicleType_unknown', 'VehicleType_wagon',
                            'Gearbox_manual', 'FuelType_electric',
                            'FuelType_gasoline', 'FuelType_hybrid',
                            'FuelType_lpg', 'FuelType_other', 'FuelType_petrol',
                            'NotRepaired_yes'],
              num_leaves=100, random_state=12345)

In [157]:
%%time
lgbm_predict = lgbmmodel.predict(features_test)

CPU times: user 789 ms, sys: 0 ns, total: 789 ms
Wall time: 801 ms


In [158]:
mse_lgbmmodel = mean_squared_error(target_test, lgbm_predict)
print("RMSE для LGBMRegressor модели на тестовой выборке:", round((mse_lgbmmodel) ** 0.5, 2))

RMSE для LGBMRegressor модели на тестовой выборке: 1650.53


<b> Были проведены измерения: </b>

- времени обучения;
- времени предсказания моделей;
- качества предсказания по метрике RMSE.

## Анализ моделей

In [ ]:
df = [["435 ms", "32.2 ms", 2892.27],
        ["16 s", "56.3 ms", 2892.26],
        ["728 ms", "13.3 ms", 2120.34],
        ["2 min 29 s", "966 ms", 1625.21],
        ["15.6 s", "902 ms", 1664.7]]
model = ["Linear Regression", "Ridge", "DecisionTreeRegresso", "CatBoostRegressor", "LGBMRegressor"]

In [ ]:
pd.DataFrame(data=df, index=model, columns=["fit_time", "predict_time", "RMSE"])

<b> Вывод: </b>
    
Наилучшее качество получилось на модели Catboost (RMSE 1625.21), время обучения было дольше остальных и составило 2 min 29 s.

LGBMRegressor предсказала незначительно хуже (RMSE 1664.70), но обучилась заметно быстрее на данных гиперпараметрах 15.6 s.

<b> Проверка модели на тестовой выборке </b>

Объединенил тренировочную и валидную выборки

In [172]:
features_train=pd.concat([features_train, features_test])
target_train=pd.concat([target_train, target_test])

In [173]:
%%time
lgbmmodel = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=100, 
                      random_state=12345,
                      cat_features=categorical_features)
lgbmmodel.fit(features_train, target_train)

CPU times: user 18.7 s, sys: 502 ms, total: 19.2 s
Wall time: 19.3 s


LGBMRegressor(cat_features=['Model', 'RegistrationMonth', 'Brand',
                            'VehicleType_convertible', 'VehicleType_coupe',
                            'VehicleType_other', 'VehicleType_sedan',
                            'VehicleType_small', 'VehicleType_suv',
                            'VehicleType_unknown', 'VehicleType_wagon',
                            'Gearbox_manual', 'FuelType_electric',
                            'FuelType_gasoline', 'FuelType_hybrid',
                            'FuelType_lpg', 'FuelType_other', 'FuelType_petrol',
                            'NotRepaired_yes'],
              num_leaves=100, random_state=12345)

In [174]:
%%time
lgbm_predict = lgbmmodel.predict(features_valid)

CPU times: user 451 ms, sys: 0 ns, total: 451 ms
Wall time: 332 ms


In [175]:
mse_lgbmmodel = mean_squared_error(target_valid, lgbm_predict)
print("RMSE для LGBMRegressor модели на тестовой выборке:", round((mse_lgbmmodel) ** 0.5, 2))

RMSE для LGBMRegressor модели на тестовой выборке: 1474.73
